In [252]:
query = "remove duplicate patientid keeping lowest age, from innerjoin




table jw.test1 where race='hi' with table jw.test2 where age > 5 on former.id = latter.idenfication"

In [278]:
def parse(simpleQuery, params={'tempidx': 0}, complexQuery=''):
    
    spaceIdx = simpleQuery.find(" ")
    function = simpleQuery[:spaceIdx]
    
    if function == 'remove':
        complexQuery += parse(simpleQuery[spaceIdx+1:], params=params)
        
    elif function == 'duplicate':
        
        simpleQuery = simpleQuery[len('duplicate '):]
        
        nextPos = simpleQuery.find(" ")
        identifier = simpleQuery[:nextPos]
        simpleQuery = simpleQuery[nextPos+1:]
        
        nextPos = simpleQuery.find(" ")
        nextWord = simpleQuery[:nextPos]
        
        if nextWord == "keeping":
            
            simpleQuery = simpleQuery[nextPos+1:]
            nextPos = simpleQuery.find(" ")
            orderWord = simpleQuery[:nextPos]
            
            if orderWord == "lowest":      
                order = 'asc'
                
            elif orderWord == "highest":        
                order = 'desc'
        
            else:
                print("Error. Cannot parse 'keeping' argument.  Last simple query: ")
                print(simpleQuery)
                return complexQuery
            
            simpleQuery = simpleQuery[nextPos+1:]
            orderSubject = simpleQuery[:simpleQuery.find(" ")]  
            
            complexQuery += '''
                            with cte as(
                            select *,
                            ROW_NUMBER() OVER (PARTITION BY {} ORDER BY age {}) as rn
                            from
                            (
                            '''.format(identifier, order)
            params['tempidx'] += 1
            complexQuery += parse(simpleQuery[spaceIdx:], params=params)
            complexQuery += '''
                            )as {}
                            )
                            select *
                            from cte
                            where rn = 1   
                            '''.format("temp"+str(params['tempidx']))
                
                

        else:
            
            complexQuery += '''
                            with cte as(
                            select *,
                            ROW_NUMBER() OVER (PARTITION BY {}) as rn
                            from
                            (
                            '''.format(identifier)
            params['tempidx'] += 1
            complexQuery += parse(simpleQuery[spaceIdx+1:], params=params)
            complexQuery += '''
                            )as {}
                            )
                            select *
                            from cte
                            where rn = 1   
                            '''.format("temp"+str(params['tempidx']))
        
    elif function == 'innerjoin':
        simpleQuery = simpleQuery[spaceIdx+1:]
        nextPos = simpleQuery.find(" ")
        identifier = simpleQuery[:nextPos]
        simpleQuery = simpleQuery[nextPos+1:]
        print(identifier)
        
        complexQuery += parse(simpleQuery[spaceIdx+1:], params=params)   
        
    elif function == 'table':
        tablePos = simpleQuery[spaceIdx+1:].find(" ")
        complexQuery += parse(simpleQuery[spaceIdx+1:], params=params)
        
    elif function == 'from': # Ignored connective word
        complexQuery += parse(simpleQuery[spaceIdx+1:], params=params)
        
    else:
        print("Error. Cannot parse. Last simple query: ")
        print(simpleQuery)

    return complexQuery

In [279]:
a = parse(query.replace(',',''))
print(a)


of
Error. Cannot parse. Last simple query: 
est1 where race='hi' with table jw.test2 where age > 5 on former.id = latter.idenfication

                            with cte as(
                            select *,
                            ROW_NUMBER() OVER (PARTITION BY patientid ORDER BY age asc) as rn
                            from
                            (
                            
                            )as temp1
                            )
                            select *
                            from cte
                            where rn = 1   
                            
